<a href="https://colab.research.google.com/github/jayantdahiya/Deep-Facial-Recognition/blob/main/Notebook1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python matplotlib

     |████████████████████████████████| 394.3 MB 14 kB/s 
     |████████████████████████████████| 394.3 MB 17 kB/s 
     |████████████████████████████████| 3.8 MB 39.6 MB/s 
     |████████████████████████████████| 2.9 MB 37.5 MB/s 
     |████████████████████████████████| 462 kB 33.7 MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68725 sha256=383a32fea8e2d2bc27207866df412d1648e5ce4d0fe6279ca164a2bbcf95e1a4
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.42.0
    Uninstalling grpcio-1.42.0:
      Successfully uninstalled grpcio-1.42.0
  Attempting uninstall: wrapt
    Found existing installat

In [3]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt

In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

In [8]:
POS_PATH = os.path.join('/content/drive/MyDrive/Colab/Deep-Facial-Recognition-Application/data', '/content/drive/MyDrive/Colab/Deep-Facial-Recognition-Application/data/positive')
NEG_PATH = os.path.join('/content/drive/MyDrive/Colab/Deep-Facial-Recognition-Application/data', '/content/drive/MyDrive/Colab/Deep-Facial-Recognition-Application/data/negative')
ANC_PATH = os.path.join('/content/drive/MyDrive/Colab/Deep-Facial-Recognition-Application/data', '/content/drive/MyDrive/Colab/Deep-Facial-Recognition-Application/data/anchor')
# os.makedirs(POS_PATH)
# os.makedirs(NEG_PATH)
# os.makedirs(ANC_PATH)

In [9]:
# for directory in os.listdir('/content/drive/MyDrive/Colab/Deep-Facial-Recognition-Application/lfw'):
#   for file in os.listdir(os.path.join('/content/drive/MyDrive/Colab/Deep-Facial-Recognition-Application/lfw', directory)):
#     EX_PATH = os.path.join('/content/drive/MyDrive/Colab/Deep-Facial-Recognition-Application/lfw', directory, file)
#     NEW_PATH = os.path.join(NEG_PATH, file)
#     os.replace(EX_PATH, NEW_PATH)